In [1]:
%cd /media/eel/DATAUbuntu/Projetos/Pub2023-2024/PortOptAggregator/

/media/eel/DATAUbuntu/Projetos/Pub2023-2024/PortOptAggregator


/media/eel/DATAUbuntu/Projetos/Pub2023-2024/PortOptAggregator/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:

import pandas as pd
#import ydata_profiling as pf
from ipywidgets import interact, widgets, fixed


mais_data  = pd.read_csv('data/processed/dados_122023.csv')
mais_data['CNPJ'] = mais_data['CNPJ do fundo']
mais_data.head(2)

fundos_ranking = pd.read_csv('data/raw/fundos_ranking_24M_202201_202312.csv', sep=';', encoding = 'ISO-8859-1')
fundos_ranking.head(2)

fundos_ranking['CNPJ'] = fundos_ranking['CNPJ_FUNDO']
fundos_ranking['CNPJ'] = pd.to_numeric( ((fundos_ranking['CNPJ'].apply(lambda x: x.replace('.',''))).apply(lambda x: x.replace('/',''))).apply(lambda x: x.replace('-','')) )
#fundos_ranking.index = fundos_ranking['CNPJ']
#fundos_ranking.drop(inplace=True, columns='CNPJ_FUNDO2')
#fundos_ranking.index.names = ['CNPJ_FUNDO']
fundos_ranking.head()


result = pd.merge(fundos_ranking, mais_data, on="CNPJ")
result.head(2)

mais_data_1M  = pd.read_csv('data/processed/dados_012024.csv', usecols=['CNPJ do fundo', 'Rentabilidade No mês'])
mais_data_1M['CNPJ'] = mais_data_1M['CNPJ do fundo']
mais_data_1M.drop(inplace=True, columns='CNPJ do fundo')

mais_data_1M['Rentabilidade 1M'] = mais_data_1M['Rentabilidade No mês']
mais_data_1M.drop(inplace=True, columns='Rentabilidade No mês')
mais_data_1M.head()

data_all = pd.merge(result, mais_data_1M, on="CNPJ")
result.head(2)

# features = ['Rentabilidade 1M','Rentabilidade No mês', 'Rentabilidade 3 meses',
#         'Rentabilidade 6 meses', 'Rentabilidade 12 meses',
#         'Rentabilidade 24 meses', 'Volatilidade No mês', 'Volatilidade 3 meses',
#         'Volatilidade 6 meses', 'Volatilidade 12 meses',
#         'Volatilidade 24 meses',
#        'Índice de sharpe No mês', 'Índice de sharpe 3 meses',
#        'Índice de Treynor No mês',
#        'Índice de Treynor 3 meses']

features = ['Rentabilidade 1M','Rentabilidade No mês', 'Rentabilidade 3 meses',
       'Rentabilidade 6 meses', 'Rentabilidade 12 meses',
       'Rentabilidade 24 meses', 'Volatilidade No mês', 'Volatilidade 3 meses',
       'Volatilidade 6 meses', 'Volatilidade 12 meses',
       'Volatilidade 24 meses', 'Beta No mês', 'Beta 3 meses', 'Beta 6 meses',
       'Beta 12 meses', 'Beta 24 meses', 'Tracking Error No mês',
       'Tracking Error 3 meses', 'Tracking Error 6 meses',
       'Tracking Error 12 meses', 'Tracking Error 24 meses',
       'Índice de sharpe No mês', 'Índice de sharpe 3 meses',
       'Índice de sharpe 6 meses', 'Índice de sharpe 12 meses',
       'Índice de sharpe 24 meses', 'Índice de sortino No mês',
       'Índice de sortino 3 meses', 'Índice de sortino 6 meses',
       'Índice de sortino 12 meses', 'Índice de sortino 24 meses',
       'Information Ratio No mês', 'Information Ratio 3 meses',
       'Information Ratio 6 meses', 'Information Ratio 12 meses',
       'Information Ratio 24 meses', 'Índice de Treynor No mês',
       'Índice de Treynor 3 meses', 'Índice de Treynor 6 meses',
       'Índice de Treynor 12 meses', 'Índice de Treynor 24 meses']

data_fundos = data_all[ features ]
data_fundos.head(2)

from itertools import combinations
import numpy as np

num_ativos = 4
num_fundos = data_fundos.shape[0]
portfolios = list( combinations(range(0, num_fundos), num_ativos) )

# número de portfólio (número de pesos aleatórios gerados)
num_portfolios = len( portfolios )

# cria um array para manter os resultados
results = np.zeros((num_portfolios, data_fundos.shape[1] ))

# cria o for loop para as simulações
for i in range(num_portfolios):
    results[i,:] = data_fundos.iloc[ list(portfolios[i]) ].mean().values
    
# Converte o resultado para um df
data = pd.DataFrame(results, index=portfolios, columns = data_fundos.columns)

In [3]:
from pycaret.regression import *
s = setup(data, target = 'Rentabilidade 1M', session_id=123)

,Description,Value
0,Session id,123
1,Target,Rentabilidade 1M
2,Target type,Regression
3,Original data shape,"(91390, 41)"
4,Transformed data shape,"(91390, 41)"
5,Transformed train set shape,"(63972, 41)"
6,Transformed test set shape,"(27418, 41)"
7,Numeric features,40
8,Preprocess,True
9,Imputation type,simple


In [5]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.0000,0.0000,0.0000,1.0000,0.0000,0.0110,0.4720
ridge,Ridge Regression,0.0003,0.0000,0.0003,0.9998,0.0003,0.0715,0.0730
et,Extra Trees Regressor,0.0003,0.0000,0.0008,0.9990,0.0008,0.0773,6.6510
rf,Random Forest Regressor,0.0005,0.0000,0.0012,0.9980,0.0011,0.1377,23.3640
dt,Decision Tree Regressor,0.0005,0.0000,0.0019,0.9950,0.0017,0.1661,0.4980
lightgbm,Light Gradient Boosting Machine,0.0016,0.0000,0.0022,0.9929,0.0021,0.4634,190.8070
knn,K Neighbors Regressor,0.0008,0.0000,0.0025,0.9915,0.0022,0.1701,0.3270
gbr,Gradient Boosting Regressor,0.0031,0.0000,0.0040,0.9774,0.0038,0.8189,10.4760
huber,Huber Regressor,0.0038,0.0000,0.0048,0.9671,0.0045,1.3223,1.1990
ada,AdaBoost Regressor,0.0065,0.0001,0.0078,0.9137,0.0072,1.7162,3.3490


In [6]:
interpret_model(best, plot = 'summary')

TypeError: This function only supports tree based models for binary classification: rf, dt, lightgbm, et.

In [7]:
tuned_rf = tune_model(rf, optimize='RMSE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0149,0.0004,0.0191,0.4805,0.0155,1.6776
1,0.0151,0.0004,0.0193,0.4739,0.0154,3.2629
2,0.0151,0.0004,0.0193,0.4799,0.0158,2.6369
3,0.0150,0.0004,0.0192,0.4803,0.0154,2.2906
4,0.0150,0.0004,0.0192,0.4770,0.0156,3.1407
5,0.0150,0.0004,0.0191,0.4772,0.0153,2.6232
6,0.0151,0.0004,0.0193,0.4799,0.0155,2.2503
7,0.0150,0.0004,0.0193,0.4729,0.0155,2.5320
8,0.0150,0.0004,0.0192,0.4765,0.0156,12.2083


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [8]:
def plot_graphs(estimator, plot):
  plot_model(estimator, plot=plot)

In [9]:
## Decidi não usar o método evaluate_model, pois existem alguns gráficos que demoram vários minutos para serem gereados e também há gráficos que não são
## suportados pelo modelo mais eficiente. Então juntei aqui os gráficos mais simples e fiz separado os que demandam mais tempo de geração.
options = ['pipeline',  'parameter',  'feature', 'error', 'manifold',   'vc' , 'residuals' , 'cooks'] #, 'tree'# criar visualização iframe]
labels = ["Pipeline","HiperParameters","Feature Importance", "Prediction Error","Manifold Learning","Validation Curve" ,
          "Residuals Plot" , "Cooks Distance Plot" #,"Decision Tree"
          ]
tb = widgets.ToggleButtons(

            description="Tipo de gráfico:",
)

interact(
            plot_graphs,
            estimator=fixed(tuned_rf),
            plot = [ (l,o) for l,o in zip(labels, options) ],
)   

interactive(children=(Dropdown(description='plot', options=(('Pipeline', 'pipeline'), ('HiperParameters', 'par…

<function __main__.plot_graphs(estimator, plot)>

In [10]:
predict_model(tuned_rf)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.0046,0.0000,0.0061,0.9481,0.0057,0.9692


,Rentabilidade No mês,Rentabilidade 3 meses,Rentabilidade 6 meses,Rentabilidade 12 meses,Rentabilidade 24 meses,Volatilidade No mês,Volatilidade 3 meses,Volatilidade 6 meses,Volatilidade 12 meses,Volatilidade 24 meses,...,Information Ratio 6 meses,Information Ratio 12 meses,Information Ratio 24 meses,Índice de Treynor No mês,Índice de Treynor 3 meses,Índice de Treynor 6 meses,Índice de Treynor 12 meses,Índice de Treynor 24 meses,Rentabilidade 1M,prediction_label
"(2, 7, 22, 24)",0.056910,0.133563,0.227259,0.617900,1.127180,0.205160,0.250328,0.245551,0.274350,0.313065,...,1.118201,1.924724,1.427551,0.111690,-0.444021,0.273558,0.382855,0.310047,0.016897,0.012955
"(4, 23, 27, 30)",0.057587,0.168936,0.181935,0.480952,0.834738,0.171833,0.213243,0.217884,0.246972,0.285153,...,0.452350,1.303967,1.019383,0.034144,-0.087823,0.104603,0.227516,0.207300,-0.015050,-0.010693
"(13, 16, 17, 33)",0.083866,0.191549,0.126158,0.283037,0.950015,0.196649,0.224053,0.218894,0.236207,0.276231,...,0.102148,0.817774,1.149976,2.677823,0.130408,-0.009188,0.036256,0.369913,0.016916,0.023839
"(1, 9, 31, 39)",0.042375,0.125929,0.212751,0.565169,1.013148,0.182632,0.226066,0.225312,0.256810,0.286810,...,0.827872,1.728532,1.147477,-0.209181,-0.408073,0.249132,0.358595,0.266344,0.023607,0.020060
"(4, 21, 31, 35)",0.049617,0.100839,0.192374,0.484932,0.795483,0.137063,0.190152,0.188941,0.203568,0.227204,...,0.848735,1.628021,1.096647,0.088950,-0.483206,0.160990,0.304314,0.225362,-0.000362,-0.000766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(11, 15, 23, 39)",0.053957,0.144370,0.189004,0.551100,0.977642,0.209392,0.258422,0.263098,0.294051,0.335152,...,0.480736,1.563101,1.054708,0.022184,-0.439817,0.142754,0.289447,0.234262,0.005462,0.012955
"(3, 11, 26, 33)",0.055505,0.130930,0.208880,0.535158,1.011642,0.176432,0.216284,0.223938,0.253920,0.283269,...,0.964588,1.722989,1.235476,0.183818,-0.397579,0.205357,0.325816,0.276393,0.020601,0.022663
"(14, 17, 19, 33)",0.123146,0.227100,0.235248,0.304516,0.835122,0.270476,0.262278,0.241228,0.233667,0.262263,...,1.073437,0.747350,0.931724,6.287129,0.729071,0.843522,0.382452,0.416898,0.034251,0.049627
"(3, 9, 10, 33)",0.043388,0.083128,0.215868,0.602849,1.191767,0.194291,0.240427,0.245612,0.275345,0.310477,...,0.852651,1.854894,1.348426,-0.112218,-0.925861,0.236205,0.394299,0.328643,0.048716,0.043050


In [11]:
final_model = finalize_model(tuned_rf)
print(final_model)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Rentabilidade No mês',
                                             'Rentabilidade 3 meses',
                                             'Rentabilidade 6 meses',
                                             'Rentabilidade 12 meses',
                                             'Rentabilidade 24 meses',
                                             'Volatilidade No mês',
                                             'Volatilidade 3 meses',
                                             'Volatilidade 6 meses',
                                             'Volatilidade 12 meses',
                                             'Volatilidade 24 meses',
                                             'Beta No mês', 'Beta 3 meses',
                                             'Beta 6 meses', '...
                                             'Índice de sortino 24 meses', ...],
   

# Teste

In [12]:
mais_data_teste  = pd.read_csv('data/processed/dados_012024.csv')
mais_data_teste['CNPJ'] = mais_data_teste['CNPJ do fundo']
fundos_ranking = pd.read_csv('data/raw/fundos_ranking_24M_202201_202312.csv', sep=';', encoding = 'ISO-8859-1')

fundos_ranking['CNPJ'] = fundos_ranking['CNPJ_FUNDO']
fundos_ranking['CNPJ'] = pd.to_numeric( ((fundos_ranking['CNPJ'].apply(lambda x: x.replace('.',''))).apply(lambda x: x.replace('/',''))).apply(lambda x: x.replace('-','')) )
#fundos_ranking.index = fundos_ranking['CNPJ']
#fundos_ranking.drop(inplace=True, columns='CNPJ_FUNDO2')
#fundos_ranking.index.names = ['CNPJ_FUNDO']
result2 = pd.merge(fundos_ranking, mais_data_teste, on="CNPJ")

mais_data_1M_teste  = pd.read_csv('data/processed/dados_022024.csv', usecols=['CNPJ do fundo', 'Rentabilidade No mês'])
mais_data_1M_teste['CNPJ'] = mais_data_1M_teste['CNPJ do fundo']
mais_data_1M_teste.drop(inplace=True, columns='CNPJ do fundo')

mais_data_1M_teste['Rentabilidade 1M'] = mais_data_1M_teste['Rentabilidade No mês']
mais_data_1M_teste.drop(inplace=True, columns='Rentabilidade No mês')

data_all2 = pd.merge(result2, mais_data_1M_teste, on="CNPJ")

data_fundos_teste = data_all2[ features ]

num_ativos = 4
num_fundos = data_fundos_teste.shape[0]
portfolios = list( combinations(range(0, num_fundos), num_ativos) )

# número de portfólio (número de pesos aleatórios gerados)
num_portfolios = len( portfolios )

# cria um array para manter os resultados
results_teste = np.zeros((num_portfolios, data_fundos_teste.shape[1] ))

# cria o for loop para as simulações
for i in range(num_portfolios):
    results_teste[i,:] = data_fundos_teste.iloc[ list(portfolios[i]) ].mean().values
    
# Converte o resultado para um df
data_teste = pd.DataFrame(results_teste, index=portfolios, columns = data_fundos_teste.columns)
data_teste.head()


# Assuming df is your DataFrame
#X_teste = data_teste[features].values  #features
#y_teste = data_teste['Rentabilidade 1M'].values  # Target variable

,Rentabilidade 1M,Rentabilidade No mês,Rentabilidade 3 meses,Rentabilidade 6 meses,Rentabilidade 12 meses,Rentabilidade 24 meses,Volatilidade No mês,Volatilidade 3 meses,Volatilidade 6 meses,Volatilidade 12 meses,...,Information Ratio No mês,Information Ratio 3 meses,Information Ratio 6 meses,Information Ratio 12 meses,Information Ratio 24 meses,Índice de Treynor No mês,Índice de Treynor 3 meses,Índice de Treynor 6 meses,Índice de Treynor 12 meses,Índice de Treynor 24 meses
"(0, 1, 2, 3)",-0.004638,0.081208,0.172983,0.369821,0.886940,1.451080,0.214328,0.220471,0.253491,0.293063,...,9.908869,1.876702,3.175140,3.000268,1.774460,2.200049,0.698366,1.319074,0.859330,0.495922
"(0, 1, 2, 4)",-0.004283,0.081228,0.174801,0.372302,0.891716,1.449956,0.212332,0.219260,0.252356,0.292799,...,9.978932,1.930232,3.209644,3.020744,1.773249,2.218231,0.717921,1.333473,0.865611,0.495163
"(0, 1, 2, 5)",-0.003742,0.081059,0.175837,0.374211,0.891299,1.450575,0.209917,0.217934,0.250868,0.291465,...,10.046459,1.964557,3.240743,3.029339,1.781171,2.235296,0.730211,1.350196,0.869081,0.495965
"(0, 1, 2, 6)",-0.003637,0.081023,0.176065,0.375008,0.892343,1.449784,0.209343,0.217719,0.250715,0.291417,...,10.063405,1.972075,3.250850,3.034365,1.780244,2.239964,0.732888,1.353867,0.870078,0.495565
"(0, 1, 2, 7)",-0.004744,0.080992,0.172440,0.368501,0.882554,1.441611,0.214176,0.220301,0.253334,0.292799,...,9.888781,1.862632,3.162827,2.984636,1.764769,2.195914,0.692708,1.314267,0.855397,0.492985


In [13]:

# previsão em dados não vistos
unseen_predictions = predict_model(final_model, data=data_teste)
unseen_predictions.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.0349,0.0025,0.0495,-0.2944,0.0407,11.5362


,Rentabilidade No mês,Rentabilidade 3 meses,Rentabilidade 6 meses,Rentabilidade 12 meses,Rentabilidade 24 meses,Volatilidade No mês,Volatilidade 3 meses,Volatilidade 6 meses,Volatilidade 12 meses,Volatilidade 24 meses,...,Information Ratio 6 meses,Information Ratio 12 meses,Information Ratio 24 meses,Índice de Treynor No mês,Índice de Treynor 3 meses,Índice de Treynor 6 meses,Índice de Treynor 12 meses,Índice de Treynor 24 meses,Rentabilidade 1M,prediction_label
"(0, 1, 2, 3)",0.081207,0.172983,0.369821,0.886940,1.451080,0.214327,0.220472,0.253491,0.293063,0.346279,...,3.175140,3.000268,1.774459,2.200049,0.698366,1.319074,0.859330,0.495922,-0.004637,0.043127
"(0, 1, 2, 4)",0.081229,0.174801,0.372302,0.891716,1.449956,0.212332,0.219260,0.252356,0.292798,0.346366,...,3.209644,3.020744,1.773249,2.218231,0.717921,1.333473,0.865611,0.495163,-0.004283,0.043127
"(0, 1, 2, 5)",0.081059,0.175837,0.374211,0.891299,1.450575,0.209917,0.217934,0.250868,0.291465,0.345104,...,3.240743,3.029339,1.781171,2.235296,0.730211,1.350196,0.869081,0.495965,-0.003741,0.043127
"(0, 1, 2, 6)",0.081023,0.176065,0.375008,0.892344,1.449784,0.209343,0.217718,0.250715,0.291417,0.345140,...,3.250850,3.034365,1.780244,2.239964,0.732888,1.353866,0.870079,0.495565,-0.003637,0.043127
"(0, 1, 2, 7)",0.080992,0.172440,0.368502,0.882554,1.441611,0.214176,0.220300,0.253334,0.292799,0.346038,...,3.162827,2.984636,1.764769,2.195914,0.692707,1.314268,0.855397,0.492985,-0.004744,0.043127


fit_kwargs: dict, default = {} (empty dict)

    Dictionary of arguments passed to the fit method of the model.
